Main resource: https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md

In [1]:
import flair
import numpy as np
import pandas as pd
import torch
from torch.optim.adam import Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
flair.device = device

print(flair.device)

cuda


In [2]:
print(torch.__version__)

1.7.1+cu110


In [3]:
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

data_folder = '../data/corpus_10042021'
column_name_map = {1: "text", 2: "label_topic"}

# 1. get the corpus
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True) 

# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()

print(label_dict)

2021-04-18 17:51:33,967 Reading data from ..\data\corpus_10042021
2021-04-18 17:51:34,016 Train: ..\data\corpus_10042021\train.csv
2021-04-18 17:51:34,017 Dev: ..\data\corpus_10042021\dev.csv
2021-04-18 17:51:34,017 Test: ..\data\corpus_10042021\test.csv
2021-04-18 17:51:34,033 Computing label dictionary. Progress:


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 976/976 [00:01<00:00, 896.52it/s]

2021-04-18 17:51:52,771 [b'High', b'Low', b'Medium']
Dictionary with 3 tags: High, Low, Medium


In [4]:
# 4. initialize document embedding by passing list of word embeddings
document_embeddings = TransformerDocumentEmbeddings('bert-base-uncased', fine_tune=True)

# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

# 6. initialize the text classifier trainer
trainer = ModelTrainer(classifier, corpus, optimizer=Adam)

# 7. start the training
trainer.train('./flair/transformers',
              learning_rate=3e-5, # use very small learning rate
              mini_batch_size=4,
              mini_batch_chunk_size=2, # optionally set this if transformer is too much for your machine
              max_epochs=10) # terminate after 10 epochs)

2021-04-18 17:52:03,566 ----------------------------------------------------------------------------------------------------
2021-04-18 17:52:03,568 Model: "TextClassifier(
  (document_embeddings): TransformerDocumentEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
               

{'test_score': 0.4862,
 'dev_score_history': [0.3889,
  0.4259,
  0.4167,
  0.3981,
  0.4074,
  0.3611,
  0.3426,
  0.3519,
  0.3889,
  0.3704],
 'train_loss_history': [1.1028372417397214,
  1.016748195694339,
  0.670027196905168,
  0.3489884364888169,
  0.09774403547540265,
  0.03974058585199527,
  0.00562146532445417,
  0.0006854604820291186,
  5.6424556055403085e-05,
  5.987358741580452e-05],
 'dev_loss_history': [1.085874319076538,
  1.0972222089767456,
  1.4012253284454346,
  3.504218816757202,
  4.503663063049316,
  4.749303817749023,
  4.91359281539917,
  4.895306587219238,
  5.066765785217285,
  5.047904014587402]}